# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [6]:
import pandas as pd

---
## 2. Load data

In [2]:
df = pd.read_csv('task_4/data/clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('task_3/data/price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

--- 

## USER Code 1

In [8]:
# Merge the new off-peak difference features into the main dataset
main_df = pd.read_csv('task_4/data/clean_data_after_eda.csv')
main_df_with_diff = main_df.merge(diff, on='id', how='left')

# Quick check
print("Shape before merge:", main_df.shape)
print("Shape after merge:", main_df_with_diff.shape)
main_df_with_diff[['offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']].head()


Shape before merge: (14606, 44)
Shape after merge: (14606, 46)


,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0.020057,3.700961
1,-0.003767,0.177779
2,-0.004670,0.177779
3,-0.004547,0.177779
4,-0.006192,0.162916


#### Step 2: Handle missing values in the new columns
Even though your output sample looks good, it's best to check for missing values across the new columns — and decide how to handle them

In [10]:
# Check for missing values in the new engineered columns
missing_summary = main_df_with_diff[['offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']].isnull().sum()
print("Missing values:\n", missing_summary)


Missing values:
 offpeak_diff_dec_january_energy    0
offpeak_diff_dec_january_power     0
dtype: int64


#### Step 3: Derive Additional Features from Dates and Interaction Terms
Now we'll generate a few more domain-informed and potentially predictive features using:

A. Date-Based Features
We'll extract components like year, month, and duration from existing date columns.

In [12]:
# Extract year and month from key date columns
main_df['date_activ'] = pd.to_datetime(main_df['date_activ'], errors='coerce')
main_df['date_end'] = pd.to_datetime(main_df['date_end'], errors='coerce')
main_df['date_modif_prod'] = pd.to_datetime(main_df['date_modif_prod'], errors='coerce')
main_df['date_renewal'] = pd.to_datetime(main_df['date_renewal'], errors='coerce')

main_df['year_renewal'] = main_df['date_renewal'].dt.year
main_df['month_renewal'] = main_df['date_renewal'].dt.month
main_df['contract_duration_days'] = (main_df['date_end'] - main_df['date_activ']).dt.days
main_df['time_to_renewal_days'] = (main_df['date_renewal'] - main_df['date_modif_prod']).dt.days


In [13]:
# Usage ratio: forecast vs actual consumption
main_df['forecast_vs_actual_ratio'] = main_df['forecast_cons_12m'] / (main_df['cons_12m'] + 1)  # +1 to avoid division by zero

# Energy margin per unit consumption
main_df['net_margin_per_unit'] = main_df['net_margin'] / (main_df['cons_12m'] + 1)

# Customer's peak price burden estimate
main_df['price_burden_peak'] = main_df['forecast_price_energy_peak'] * main_df['cons_12m']


In [14]:
# Show updated shape
print("Shape after new features:", main_df.shape)

# Preview the new columns
main_df[['year_renewal', 'month_renewal', 'contract_duration_days', 'time_to_renewal_days',
         'forecast_vs_actual_ratio', 'net_margin_per_unit', 'price_burden_peak']].head()


Shape after new features: (14606, 51)


,year_renewal,month_renewal,contract_duration_days,time_to_renewal_days,forecast_vs_actual_ratio,net_margin_per_unit,price_burden_peak
0,2015,6,1096,-131,0.000000,678.990000,0.000000
1,2015,8,2566,2201,0.040753,0.004053,0.000000
2,2015,4,2192,1827,0.088000,0.012110,47.817056
3,2015,3,2192,1827,0.151445,0.016063,0.000000
4,2015,3,2245,1881,0.100712,0.010840,442.566375


In [18]:
main_df['tenure_days'] = (main_df['date_end'] - main_df['date_activ']).dt.days
main_df['is_late_renewal'] = (main_df['time_to_renewal_days'] < 0).astype(int)
main_df

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,churn,year_renewal,month_renewal,contract_duration_days,time_to_renewal_days,forecast_vs_actual_ratio,net_margin_per_unit,price_burden_peak,tenure_days,is_late_renewal
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,1,2015,6,1096,-131,0.000000,678.990000,0.000000,1096,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0,2015,8,2566,2201,0.040753,0.004053,0.000000,2566,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0,2015,4,2192,1827,0.088000,0.012110,47.817056,2192,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0,2015,3,2192,1827,0.151445,0.016063,0.000000,2192,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0,2015,3,2245,1881,0.100712,0.010840,442.566375,2245,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,18463073fb097fc0ac5d3e040f356987,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,2015-05-08,2014-05-26,4648.01,...,0,2014,5,1445,-347,0.144031,0.011830,0.000000,1445,1
14602,d0a6f71671571ed83b2645d23af6de00,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2015-08-28,631.69,...,1,2015,8,1461,1096,0.087443,0.012506,663.735916,1461,0
14603,10e6828ddd62cbcf687cb74928c4c2d2,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,2012-02-08,2015-02-09,190.39,...,1,2015,2,1460,1097,0.103192,0.011046,184.427660,1460,0
14604,1cf20fd6206d7678d5bcafd28c53b4db,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,2012-08-30,2015-08-31,19.34,...,0,2015,8,1461,1096,0.146515,0.007273,0.000000,1461,0
